# Catalogue and Particle Reading Scripts
### This notebook contains the scripts for obtaining both catalogue data and particle data.

## Catalogue Read Script

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import numpy as np
import pyread_eagle
import matplotlib.pyplot as plt
from pyread_eagle import EagleSnapshot
import os
import h5py as h5

h = 0.6727

In [1]:
def catalogue_read(table,
                   quantity,
                   data_location,
                   sim = 'Organic',
                   model = 'RECAL',
                   tag = '028_z000p000',
                   phys_units=True,
                   cgs_units=False,                     
                   verbose=False):

    # Do a quick check to make sure a valid table has been specified
    assert table in ['FOF','Subhalo'],'table must be either FOF or Subhalo'

    subfindfile_pattern = os.path.join(data_location , model , sim , f'groups_{tag}', f'eagle_subfind_tab_{tag}.{{}}.hdf5')
    
    file_ind = 0
    data_arr = None
    dt = None
    while True:

        try:
            with h5.File(subfindfile_pattern.format(file_ind), 'r') as f:
                try:
                    
                    data = f['/'+table+'/'+quantity]

                    if file_ind == 0:

                        h_scale_exponent = data.attrs['h-scale-exponent']
                        a_scale_exponent = data.attrs['aexp-scale-exponent']
                        cgs_conversion_factor = data.attrs['CGSConversionFactor']
                        h = f['Header'].attrs['HubbleParam']
                        a = f['Header'].attrs['ExpansionFactor']

                        # Let's only print this stuff out if the user wants it!
                        if verbose:
                            print('Loading ',quantity)
                            print('h exponent = ',h_scale_exponent)
                            print('a exponent = ',a_scale_exponent)
                            print('cgs conversion factor = ',cgs_conversion_factor)
                            print('h = ',h)
                            print('a = ',a)

                        # Lets be a bit more cautious and make sure we cast our HDF5 dataset into an array of the correct type
                        dt = data.dtype
                        
                        data_arr = np.array(data, dtype = dt)
                    
                    data_arr = np.append(data_arr,np.array(data,dtype=dt),axis=0)

                        
                except KeyError: 
                    pass
                        
        except OSError:
            if verbose:
                print('Run out of files after loading ',file_ind)
            break

        file_ind += 1
    
    # If the data we're loading is integer-type, no corrections will be needed
    if np.issubdtype(dt,np.integer):
        return data_arr

    # Otherwise, do unit corrections
    else:

        if phys_units:
            h = 0.6727
            h_scale_exponent = -1
            a = 1
            a_scale_exponent = 1
            data_arr *= np.power(h,h_scale_exponent) * np.power(a,a_scale_exponent)

        if cgs_units:

            # cgs numbers can be huge and overflow np.float32
            # Recast the data to float64 to be safe
            data_arr = np.array(data_arr,dtype=np.float64)

            data_arr *= cgs_conversion_factor

        return data_arr


## Particle Read Script

In [2]:
def make_snapshot_template(data_location, model, sim, tag): 
    return os.path.join(data_location , model , sim , f'particledata_{tag}', f'eagle_subfind_particles_{tag}.{{}}.hdf5')
    

def particle_read(ptype,
                  quantity,
                  lower_bounds,
                  upper_bounds,
                  data_location,
                  snapshot,
                  centre=None,
                  sim = 'Organic',
                  model = 'RECAL',
                  tag = '028_z000p000',
                  phys_units=True,
                  cgs_units=False,
                  verbose=False):
    
    # The arguments are xmin,xmax,ymin,ymax,zmin,zmax
    snapshot.select_region(lower_bounds[0], upper_bounds[0], lower_bounds[1], upper_bounds[1], lower_bounds[2], upper_bounds[2])


    snapshot_pattern = make_snapshot_template(data_location , model , sim , tag)
    data_arr = None
    dt = None
    # Initialise an index
    snapfile_ind = 0
    while True:
        try:
            with h5.File(snapshot_pattern.format(snapfile_ind), 'r') as f:
                # Grab all the conversion factors from the header and dataset attributes
                h = f['Header'].attrs['HubbleParam']
                a = f['Header'].attrs['ExpansionFactor']
                h_scale_exponent = f['/PartType%i/%s'%((ptype,quantity))].attrs['h-scale-exponent']
                a_scale_exponent = f['/PartType%i/%s'%((ptype,quantity))].attrs['aexp-scale-exponent']
                cgs_conversion_factor = f['/PartType%i/%s'%((ptype,quantity))].attrs['CGSConversionFactor']
        except:
            if verbose:
                # If there are no particles of the right type in chunk 0, move to the next one
                print('No particles of type ',ptype,' in snapfile ',snapfile_ind)
            snapfile_ind += 1
            continue
        # If we got what we needed, break from the while loop
        break

    # Load in the quantity
    data_arr = snapshot.read_dataset(ptype,quantity)

    # Cast the data into a numpy array of the correct type
    dt = data_arr.dtype
    data_arr = np.array(data_arr,dtype=dt)
    x = len(data_arr)
    if data_arr.shape == (x,3):
        
        data_arr = data_arr[(data_arr[:,0] >= lower_bounds[0]) & (data_arr[:,0] <= upper_bounds[0]) & (data_arr[:,1] >= lower_bounds[1]) & (data_arr[:,1] <= upper_bounds[1]) & (data_arr[:,2] >= lower_bounds[2]) & (data_arr[:,2] <= upper_bounds[2])]

        
#    if centre is not None:
#        #Handling the periodic box (optionally)
#        data_arr = ((data_arr - centre + (boxsize/2)) % boxsize) - (boxsize/2)

    if not np.issubdtype(dt,np.integer):
        # Do unit corrections, like we did for the catalogues
        if phys_units:
            data_arr *= np.power(h,h_scale_exponent) * np.power(a,a_scale_exponent)

        if cgs_units:
            data_arr = np.array(data_arr,dtype=np.float64)
            data_arr *= cgs_conversion_factor
  
        

    return data_arr


def make_snapshot_template(data_location, model, sim, tag): 
    return os.path.join(data_location , model , sim , f'particledata_{tag}', f'eagle_subfind_particles_{tag}.{{}}.hdf5')
    

def particle_read_sphere(ptype,
                  quantity,
                  centre=None,
                  radius,
                  data_location,
                  snapshot,
                  sim = 'Organic',
                  model = 'RECAL',
                  tag = '028_z000p000',
                  phys_units=True,
                  cgs_units=False,
                  verbose=False):
    
    # The arguments are xmin,xmax,ymin,ymax,zmin,zmax
    snapshot.select_region(lower_bounds[0], upper_bounds[0], lower_bounds[1], upper_bounds[1], lower_bounds[2], upper_bounds[2])


    snapshot_pattern = make_snapshot_template(data_location , model , sim , tag)
    
    # Initialise an index
    snapfile_ind = 0
    while True:
        try:
            with h5.File(snapshot_pattern.format(snapfile_ind), 'r') as f:
                # Grab all the conversion factors from the header and dataset attributes
                h = f['Header'].attrs['HubbleParam']
                a = f['Header'].attrs['ExpansionFactor']
                h_scale_exponent = f['/PartType%i/%s'%((ptype,quantity))].attrs['h-scale-exponent']
                a_scale_exponent = f['/PartType%i/%s'%((ptype,quantity))].attrs['aexp-scale-exponent']
                cgs_conversion_factor = f['/PartType%i/%s'%((ptype,quantity))].attrs['CGSConversionFactor']
        except:
            if verbose:
                # If there are no particles of the right type in chunk 0, move to the next one
                print('No particles of type ',ptype,' in snapfile ',snapfile_ind)
            snapfile_ind += 1
            continue
        # If we got what we needed, break from the while loop
        break

    # Load in the quantity
    data_arr = snapshot.read_dataset(ptype,quantity)

    # Cast the data into a numpy array of the correct type
    dt = data_arr.dtype
    data_arr = np.array(data_arr,dtype=dt)
    
    data_arr = data_arr[(data_arr[:,0] >= lower_bounds[0]) & (data_arr[:,0] <= upper_bounds[0]) & (data_arr[:,1] >= lower_bounds[1]) & (data_arr[:,1] <= upper_bounds[1]) & (data_arr[:,2] >= lower_bounds[2]) & (data_arr[:,2] <= upper_bounds[2])]
    
#    if centre is not None:
#        #Handling the periodic box (optionally)
#        data_arr = ((data_arr - centre + (boxsize/2)) % boxsize) - (boxsize/2)

    if not np.issubdtype(dt,np.integer):
        # Do unit corrections, like we did for the catalogues
        if phys_units:
            data_arr *= np.power(h,h_scale_exponent) * np.power(a,a_scale_exponent)

        if cgs_units:
            data_arr = np.array(data_arr,dtype=np.float64)
            data_arr *= cgs_conversion_factor
  
        

    return data_arr